In [1]:
import cv2
import numpy as np

In [2]:
boundary_x = 1000

In [3]:
cap = cv2.VideoCapture('../resources/input/in_out_long.mov')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
back_sub_alg = cv2.createBackgroundSubtractorMOG2()
frame_height

1080

In [4]:
while True:
    hand_in = False
    ret, frame = cap.read()
    if not ret:
        break
    back_sub_mask = back_sub_alg.apply(frame)
    kernel = np.ones((6, 6),np.uint8)
    back_sub_mask = cv2.morphologyEx(back_sub_mask, cv2.MORPH_OPEN, kernel)
    _, binary_mask = cv2.threshold(back_sub_mask, 250, 255, cv2.THRESH_BINARY)
    if np.sum(binary_mask[:, :boundary_x]==255) > 100_000:
        hand_in = True
    annotated = cv2.line(frame, (boundary_x, 0), (boundary_x, frame_height), (255, 255, 0), 3)
    text = "Hand In" if hand_in else "Hand Out"
    cv2.imshow("Frame", cv2.putText(frame, text, (50, 100), 0, 3, (255, 255, 255)))
    cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

    

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
37
54
309
2832
24624
58599
96616
129004
159441
195257
226312
249213
276426
316756
331295
344032
337643
347076
361293
395744
389619
393773
415677
448750
471364
489566
506476
512182
534458
550368
567651
589151
586213
567993
551282
542509
543490
545891
539787
524536
511496
506816
502259
474931
435956
405802
388012
362438
345683
332003
303020
290920
282770
265739
246762
232825
211066
195881
180386
155407
135256
121389
109119
106868
98714
92263
79667
76116
69186
74619
78940
94845
106896
117810
120962
133114
142401
149122
142372
143569
138313
139880
139997
145928
151919
175982
230919
286051
293063
271662
222563
305402
372827
400864
434529
486046
516962
512018
485949
436222
388502
325533
257997
91904
45043
17138
11975
8154
6716
6268
5927
7548
7650
6346
1705
3947
854
931
1023
506
345
260
181
162
0
42
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
